# Model 1
## Malaria Tespiti için Derin Öğrenme Modeli


CNN With Custom Images

The dataset contains 2 folders
Infected
Uninfected
And a total of 27,558 images.

Please get this Dataset from the official NIH Website: https://ceb.nlm.nih.gov/repositories/malaria-datasets/

or

https://www.kaggle.com/datasets/iarunava/cell-images-for-detecting-malaria

In [8]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, InputLayer, Reshape, MaxPooling2D, Flatten, Rescaling

In [5]:
import tensorflow as tf
print(tf.__version__)

2.16.1


In [6]:
# Define the data directory
data_dir = "/kaggle/input/cell-images-for-detecting-malaria/cell_images/cell_images"

# Load the training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir, # Path to the data directory
    image_size=(128, 128),  # Görüntüleri 128x128 piksel boyutuna yeniden boyutlandır
    subset='training',
    batch_size=16,  # Her bir batch'teki görüntü sayısı
    shuffle=True, # Verileri rastgele karıştır
    seed=123, # Random seed for reproducibility
    validation_split=0.2
)

# Load the validation dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir, # Path to the data directory
    image_size=(128, 128),  # Reduced image size
    subset='validation', # Specify this as the validation dataset
    batch_size=16,  # Her bir batch'teki görüntü sayısı
    validation_split=0.2,
    seed=123 # Random seed for reproducibility
)

Found 27558 files belonging to 2 classes.
Using 22047 files for training.
Found 27558 files belonging to 2 classes.
Using 5511 files for validation.


In [9]:
# Normalize pixel values between 0 and 1
normalization_layer = Rescaling(1./255) # Piksel değerlerini normalleştirmek için Rescaling katmanı

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) # Map normalization to images in the training dataset
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

In [10]:
model = Sequential([
    InputLayer(input_shape=(128, 128, 3)),  # Girdi katmanı, görüntülerin boyutunu (128x128 piksel ve 3 renk kanalı) belirtir
    Conv2D(12, (3, 3), activation="relu"),  # İlk evrişim katmanı, 12 filtre ve 3x3 kernel boyutu ile, ReLU aktivasyon fonksiyonu kullanır
    MaxPooling2D(pool_size=(2, 2)),  # Max pooling katmanı, görüntüyü küçültmek için 2x2 havuzlama kullanır
    Conv2D(64, (3, 3), activation="relu"),  # İkinci evrişim katmanı, 64 filtre ve 3x3 kernel boyutu ile, ReLU aktivasyon fonksiyonu kullanır
    Flatten(),  # Çıktıyı 2 boyutludan 1 boyutlu hale getirir (fully connected layer için)
    Dense(10, activation="relu"),  # Fully connected katman, 10 nöron ve ReLU aktivasyon fonksiyonu ile
    Dense(1, activation="sigmoid")  # Çıkış katmanı, 1 nöron ve sigmoid aktivasyon fonksiyonu ile (ikili sınıflandırma için)
])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [11]:
# Modeli Adam optimizasyonu, binary crossentropy kaybı ve accuracy metriği ile derle
model.compile(optimizer="adam",  # Adam optimizörü
              loss=tf.keras.losses.BinaryCrossentropy(),  # Binary crossentropy kaybı (ikili sınıflandırma için)
              metrics=["accuracy"])  # Eğitim sırasında accuracy metriğini takip et

In [12]:
history2 = model.fit(train_ds, # Training dataset
                     epochs=5, # Number of training epochs
                     validation_data=val_ds # Validation dataset
                    )

Epoch 1/5
1378/1378 ━━━━━━━━━━━━━━━━━━━━ 164s 117ms/step - accuracy: 0.4990 - loss: 0.7408 - val_accuracy: 0.4916 - val_loss: 0.6933
Epoch 2/5
1378/1378 ━━━━━━━━━━━━━━━━━━━━ 199s 116ms/step - accuracy: 0.5002 - loss: 0.6932 - val_accuracy: 0.4916 - val_loss: 0.6933
Epoch 3/5
1378/1378 ━━━━━━━━━━━━━━━━━━━━ 159s 116ms/step - accuracy: 0.5005 - loss: 0.6932 - val_accuracy: 0.4916 - val_loss: 0.6933
Epoch 4/5
1378/1378 ━━━━━━━━━━━━━━━━━━━━ 159s 116ms/step - accuracy: 0.5006 - loss: 0.6932 - val_accuracy: 0.4916 - val_loss: 0.6933
Epoch 5/5
1378/1378 ━━━━━━━━━━━━━━━━━━━━ 202s 116ms/step - accuracy: 0.4987 - loss: 0.6932 - val_accuracy: 0.4916 - val_loss: 0.6933


In [13]:
# Evaluate the model on the validation dataset to check its performance
test_loss, test_accuracy = model.evaluate(val_ds)  # Evaluate the model on validation data
print(f"Test Loss: {test_loss}")  # Print the test loss
print(f"Test Accuracy: {test_accuracy}")  # Print the test accuracy

345/345 ━━━━━━━━━━━━━━━━━━━━ 12s 35ms/step - accuracy: 0.4903 - loss: 0.6933
Test Loss: 0.6932913661003113
Test Accuracy: 0.4915623366832733
